# Week 5
## Shakhyry Walsh
### IS 362
#### Professor Larry Cohen

In [5]:
airports = pd.read_csv(DATA_DIR / "airports.csv")
weather  = pd.read_csv(DATA_DIR / "weather.csv")

airports.shape, weather.shape



((1458, 8), (26115, 15))

## Data files
airports.csv, weather.csv, airlines.csv, planes.csv

In [6]:
airports.head(3), weather.head(3)


(   faa                           name       lat        lon   alt  tz dst  \
 0  04G              Lansdowne Airport 41.130472 -80.619583  1044  -5   A   
 1  06A  Moton Field Municipal Airport 32.460572 -85.680028   264  -6   A   
 2  06C            Schaumburg Regional 41.989341 -88.101243   801  -6   A   
 
               tzone  
 0  America/New_York  
 1   America/Chicago  
 2   America/Chicago  ,
   origin  year  month  day  hour      temp      dewp     humid   wind_dir  \
 0    EWR  2013      1    1     1 39.020000 26.060000 59.370000 270.000000   
 1    EWR  2013      1    1     2 39.020000 26.960000 61.630000 250.000000   
 2    EWR  2013      1    1     3 39.020000 28.040000 64.430000 240.000000   
 
    wind_speed  wind_gust   precip    pressure     visib             time_hour  
 0   10.357020        NaN 0.000000 1012.000000 10.000000  2013-01-01T06:00:00Z  
 1    8.055460        NaN 0.000000 1012.300000 10.000000  2013-01-01T07:00:00Z  
 2   11.507800        NaN 0.000000 1012.

In [7]:
us_tzones = {
    "America/Anchorage",
    "America/Chicago",
    "America/Denver",
    "America/Los_Angeles",
    "America/New_York",
    "America/Phoenix",
    "Pacific/Honolulu",
}
us_airports = airports[airports["tzone"].isin(us_tzones)].copy()
len(us_airports), us_airports.head(3)


(1451,
    faa                           name       lat        lon   alt  tz dst  \
 0  04G              Lansdowne Airport 41.130472 -80.619583  1044  -5   A   
 1  06A  Moton Field Municipal Airport 32.460572 -85.680028   264  -6   A   
 2  06C            Schaumburg Regional 41.989341 -88.101243   801  -6   A   
 
               tzone  
 0  America/New_York  
 1   America/Chicago  
 2   America/Chicago  )

In [8]:
top_north = (
    us_airports.sort_values("lat", ascending=False)
    [["faa","name","lat","lon","tzone"]]
    .head(5)
    .reset_index(drop=True)
)
top_north


,faa,name,lat,lon,tzone
0,BRW,Wiley Post Will Rogers Mem,71.285446,-156.766003,America/Anchorage
1,AIN,Wainwright Airport,70.638056,-159.994722,America/Anchorage
2,K03,Wainwright As,70.613378,-159.860350,America/Anchorage
3,ATK,Atqasuk Edward Burnell Sr Memorial Airport,70.467300,-157.436000,America/Anchorage
4,UUK,Ugnu-Kuparuk Airport,70.330833,-149.597500,America/Anchorage


## 1. What is the northernmost airport in the United States?

**Answer, Wiley Post Will Rogers Memorial Airport, BRW, Utqiagvik Alaska, latitude about 71.285 N**

**Why** I filtered airports to United States time zones, then sorted by latitude and reviewed the top five candidates, BRW appears first

**Assumption**
Time zone is a practical proxy for United States and territories in this dataset

In [9]:
top_east = (
    us_airports.sort_values("lon", ascending=False)
    [["faa","name","lat","lon","tzone"]]
    .head(5)
    .reset_index(drop=True)
)
top_east


,faa,name,lat,lon,tzone
0,SYA,Eareckson As,52.712275,174.113620,America/Anchorage
1,EPM,Eastport Municipal Airport,44.910111,-67.012694,America/New_York
2,HUL,Houlton Intl,46.123083,-67.792056,America/New_York
3,CAR,Caribou Muni,46.871500,-68.017917,America/New_York
4,PQI,Northern Maine Rgnl At Presque Isle,46.688958,-68.044797,America/New_York


## 2. What is the easternmost airport in the United States?

**Answer, Eareckson Air Station, SYA, Shemya Island Alaska, longitude about 174.114 E**

**Why**
I used the same United States filter, then sorted by longitude descending, positive values are east, Aleutian locations cross the 180th meridian, making SYA the easternmost by this definition

**Assumption**
Easternmost means greatest east longitude on the minus 180 to plus 180 scale, United States territories and military airfields are in scope

In [10]:
from pandas import NA

w = weather[(weather["year"]==2013) & (weather["month"]==2) & (weather["day"]==12)].copy()

# keep suspicious rows for the writeup
suspect = w[w["wind_speed"] > 100][["origin","hour","wind_speed","wind_gust"]].sort_values("wind_speed", ascending=False)

# remove impossible wind speeds
w.loc[w["wind_speed"] > 100, "wind_speed"] = NA

summary = (
    w.groupby("origin", as_index=True)
     .agg(
         hours=("hour","count"),
         mean_wind_speed=("wind_speed","mean"),
         max_wind_speed=("wind_speed","max"),
         mean_wind_gust=("wind_gust","mean"),
         max_wind_gust=("wind_gust","max"),
     )
     .sort_values("mean_wind_speed", ascending=False)
)
suspect, summary


(     origin  hour  wind_speed  wind_gust
 1009    EWR     3 1048.360580        NaN,
         hours  mean_wind_speed  max_wind_speed  mean_wind_gust  max_wind_gust
 origin                                                                       
 LGA        24        14.960140       23.015600       24.885618      31.071060
 JFK        24        14.384750       20.714040       24.584845      27.618720
 EWR        24        13.258987       21.864820       25.173313      31.071060)

## 3. On February 12th, 2013, which New York area airport had the windiest weather?

**Answer, LaGuardia, LGA, highest average wind speed for that date**

**Method**
I filtered weather.csv to the date, summarized mean wind speed and gusts by airport, I flagged any wind speed greater than 100 as missing, one extreme value at EWR appears impossible

**Outlier note**
A single EWR wind speed above one thousand would inflate averages, removing it aligns the three airports with realistic values, LGA remains highest by mean wind speed, daily peak gusts were about 31 miles per hour at LGA and EWR


In [11]:
top_north.to_csv("candidates_northernmost.csv", index=False)
top_east.to_csv("candidates_easternmost.csv", index=False)
summary.reset_index().to_csv("ny_wind_2013_02_12.csv", index=False)


## Files produced in this notebook
candidates_northernmost.csv, candidates_easternmost.csv, ny_wind_2013_02_12.csv